In [1]:
# 全局设置
import sys
import datetime as dt

import numpy as np
import pandas as pd

import QuantStudio.api as QS
fd = QS.FactorDB.FactorTools

HDB = QS.FactorDB.HDF5DB(config_file="../config/HDF5DBConfig.json").connect();

In [2]:
# 参数设置
StartDT, EndDT = dt.datetime(2014, 1, 1), dt.datetime(2019, 4, 30)# 数据起止时间
TestStartDT, TestEndDT = dt.datetime(2018, 1, 1), EndDT# 测试起止时间


FT = HDB.getTable("stock_cn_day_bar_nafilled")
DTs = FT.getDateTime(start_dt=StartDT, end_dt=EndDT)
TestDTs = FT.getDateTime(start_dt=TestStartDT, end_dt=TestEndDT)
IDs = FT.getID()

# 再平衡时点序列
# BalanceDTs = DTs
BalanceDTs = QS.Tools.DateTime.getMonthLastDateTime(DTs)# 月末
# BalanceDTs = QS.Tools.DateTime.getWeekLastDateTime(DTs)# 周末
# BalanceDTs = QS.Tools.DateTime.getQuarterLastDateTime(DTs)# 季末
# BalanceDTs = QS.Tools.DateTime.getYearLastDateTime(DTs)# 年末

In [3]:
# 多模块对比测试
# 创建自定义因子表
CFT = QS.FactorDB.CustomFT("CFT")
CFT.Args["遍历模式"]["向后缓冲时点数"] = 60
FT = HDB.getTable("stock_cn_day_bar_nafilled")
TargetFactors = ["high", "low"]
CFT.addFactors(factor_table=FT, factor_names=TargetFactors)

# FT = HDB.getTable("stock_cn_day_bar_nafilled")
# CFT.addFactors(factor_table=FT, factor_names=["if_trading"])

FT = HDB.getTable("stock_cn_day_bar_nafilled")
CFT.addFactors(factor_table=FT, factor_names=["close"])

CFT.setDateTime(DTs)
CFT.setID(IDs)

# 创建回测模型
Model = QS.BackTest.BackTestModel()

# --------因子测试模块--------
iModule = QS.BackTest.MultiModule()
for i, iFactorName in enumerate(TargetFactors):
    # IC 的衰减
    iSubModule = QS.BackTest.SectionFactor.ICDecay(name=iFactorName, factor_table=CFT)
    iSubModule.Args["筛选条件"] = ""
    iSubModule.Args["测试因子"] = iFactorName
    iSubModule.Args["排序方向"] = "升序"
    iSubModule.Args["价格因子"] = "close"
    iSubModule.Args["类别因子"] = "无"
    iSubModule.Args["权重因子"] = "等权"
    iSubModule.Args["计算时点"] = BalanceDTs
    iSubModule.Args["回溯期数"] = np.arange(1,13).tolist()
    iSubModule.Args["相关性算法"] = "spearman"
    iModule.Modules.append(iSubModule)
Model.Modules.append(iModule)

# 运行模型
Model.run(dts=TestDTs, subprocess_num=0)

# 查看结果
# display(Model)
from QSExt.GUI.Notebook import showOutput
Dlg = showOutput(data=Model.output())